This notebook is part of the supplementary material of the books "Online Machine Learning - Eine praxisorientiere Einführung",  
https://link.springer.com/book/9783658425043 and "Online Machine Learning - A Practical Guide with Examples in Python" https://link.springer.com/book/9789819970063
The contents are open source and published under the "BSD 3-Clause License".
This software is provided "as is" without warranty of any kind, either express or implied, including but not limited to implied warranties of merchantability and fitness for a particular purpose. The author or authors assume no liability for any damages or liability, whether in contract, tort, or otherwise, arising out of or in connection with the software or the use or other dealings with the software.

# Chapter 2: Supervised Learning: Classification and Regression

# Linear online regression with River

* Linear online regression can be performed in `river` with the `LinearRegression` class 
from the `linear_model` module. 
* In the following example, the MAE (mean absolute error) error is measured during training.

## Laden der Daten

* First, we create a dataset for classification using the `sklearn.datasets` class `make_classification`.

In [ ]:
## Generate a classification dataset with sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=10000, n_features=20, n_informative=2, n_redundant=2, random_state=42)

## Convert X to a pandas dataframe
import pandas as pd
X = pd.DataFrame(X)
y = pd.Series(y)


## Metric Selection and Data Preprocessing

* The Mean Absolute Error (MAE) is selected as the metric.
* In addition, the scaling of the data is specified:
  * By selecting the `StandardScalers` class, the data is scaled to have zero mean and one variance.
  * Internally, a running mean and running variance are determined. This scaling differs slightly from the scaling of the data in the batch because the exact means and variances are not known in advance.

In [ ]:
from river import metrics
from river import preprocessing

metric = metrics.MAE()
scaler = preprocessing.StandardScaler()

## Selection of the model

* In the next step the model (here the `LinearRegression`) is selected.
* For the sequential online optimization of the model coefficients, the stochastic gradient descent (`SGD`) is selected.
  * The learning rate for `SGD` is set to 0.01. 
  * The setting of a suitable learning rate is crucial for the optimization.

In [ ]:
from river import optim
from river import linear_model

optimizer = optim.SGD(lr=0.01)
lin_reg = linear_model.LinearRegression(optimizer)

# Pipelines

* The following example is based on [https://riverml.xyz/latest/recipes/pipelines/](https://riverml.xyz/latest/recipes/pipelines/).
* The compose.Pipeline contains all the logic for building and applying pipelines.
* A pipeline is essentially a list of estimators that are applied in sequence.
* The only requirement is that the first n - 1 steps be transformers.
* The last step can be a regressor, a classifier, a clusterer, a transformer, etc. Here is an example:

In [ ]:
from river import compose
from river import linear_model
from river import preprocessing
from river import feature_extraction

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LinearRegression()
)

* The model can be visualized:

In [ ]:
print(model)

In [ ]:
model

* `compose.Pipeline` inherits from `base.Estimator` and thus has a `learn_one` method.
* Before river v.0.19.0, the `learn_one` method updated the supervised parts of the pipeline.
* This behavior change in river v.0.19.0, where the `learn_one` method now updates the entire pipeline.
* The `predict_one` method updated the unsupervised parts of the pipeline (and not the supervised parts?).
* Let's see how this works in the following example.

In [ ]:
from river import datasets

dataset = datasets.TrumpApproval()
x, y = next(iter(dataset))
x, y

* Take a look at the unsupervised parts of the pipeline, i.e. the `StandardScaler`:

In [ ]:
model['StandardScaler'].means

* The unsupervised parts of the pipeline empty, because no data has been passed through the pipeline yet.
* If we call the `predict_one` method, only the unsupervised parts of the pipeline are updated, but not the model (supervised parts).

In [ ]:
model.predict_one(x)

In [ ]:
model['StandardScaler'].means

* As we can see, the means of each feature have been updated (they were standardized and have a zero mean), even though we called `predict_one` and not `learn_one`.
* Now let's take a look at the supervised parts of the pipeline, i.e., the model and its weights:

In [ ]:
model["LinearRegression"].weights

* Since we have called `predict_one` and not `learn_one`, the model has not been updated.
* The model weights are empty, as expected.

* We can even call `transform_one` with this model (that does not have any weights) and it will work, because the output from the last transformer (which is thus the penultimate step) will be returned in this case:

In [ ]:
model.transform_one(x)

* Calling `transform_one` does not update the model weights, as expected:

In [ ]:
model["LinearRegression"].weights

* However, usually we want to update the model weights, so we should call `learn_one`:

In [ ]:
model.learn_one(x, y)
model["LinearRegression"].weights

* Finally, after calling `learn_one`, the model weights have been updated.

In [ ]:
from river import datasets

dataset = datasets.Bananas()
x, y = next(iter(dataset))
x, y

In [ ]:
from river import linear_model
from river import optim
from river import preprocessing
from river import stream
from river import metrics
from sklearn.metrics import roc_auc_score

scaler = preprocessing.StandardScaler()
optimizer = optim.SGD(lr=0.01)
log_reg = linear_model.LogisticRegression(optimizer)

y_true = []
y_pred = []

for xi, yi in dataset:

    # Scale the features
    scaler.learn_one(xi)
    xi_scaled = scaler.transform_one(xi)

    # Test the current model on the new "unobserved" sample
    yi_pred = log_reg.predict_proba_one(xi_scaled)
    # Train the model with the new sample
    log_reg.learn_one(xi_scaled, yi)

    # Store the truth and the prediction
    y_true.append(yi)
    y_pred.append(yi_pred[True])

print(f"y: {y_true[:5]}")
print(f"y_pred: {y_pred[:5]}")
# print(f'ROC AUC: {metrics.Accuracy(y_true, y_pred):.3f}')
print(f'ROC AUC: {roc_auc_score(y_true, y_pred):.3f}')

# Metric and metric.update()

In [ ]:
from river import metrics
from river import datasets

dataset = datasets.Phishing()
model = linear_model.LogisticRegression()
metric = metrics.ROCAUC()
for x, y in dataset:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    print(f"y: {y}, y_pred: {y_pred}")
    metric.update(y, y_pred)

metric

## Test-then-train 

* Now the single samples of the dataset are used for testing and training. 
  * For each sample, the metric is updated.
* Finally, the metric that was incrementally calculated on the data from the entire process is available.

In [ ]:
## Generate a classification dataset with sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1_000, n_features=3, n_informative=2, n_redundant=1, random_state=42)

## Convert X to a pandas dataframe
import pandas as pd
X = pd.DataFrame(X)
# convert y to boolean
y = pd.Series(y).astype(bool)
y = pd.Series(y)
print(X.head())
print(y.head())

In [ ]:
from river import stream
from river import linear_model
from river import preprocessing
from river import metrics

# scaler = preprocessing.StandardScaler()
scaler = preprocessing.MinMaxScaler()
log_reg = linear_model.LogisticRegression()
metric = metrics.ROCAUC()

y_true = []
y_pred = []
metric_list = []

for xi, yi in stream.iter_pandas(X, y):
    
    print(f"xi: {xi}")
    
    # Before river 0.21.0:
    # Scale the features
    #xi_scaled = scaler.learn_one(xi).transform_one(xi)

    # After 0.21.0:
    scaler.learn_one(xi)
    xi_scaled = scaler.transform_one(xi)
    
    print(f"xi_scaled: {xi_scaled}")
    
    # Test the current model on the new "unobserved" sample
    yi_pred = log_reg.predict_proba_one(xi_scaled)
    # yi_pred = log_reg.predict_one(xi_scaled)
    print(f"yi: {yi}, yi_pred: {yi_pred}")

    # Train the model with the new sample
    log_reg.learn_one(xi_scaled, yi)
    
    metric.update(yi, yi_pred)
    print(f"metric: {metric}")
    # Store the metric after each sample in a list
    metric_list.append(metric.get())
    
    # Store the truth and the prediction
    y_true.append(yi)
    y_pred.append(yi_pred)
    print(f"y: {y_true[:5]}")
    print(f"y_pred: {y_pred[:5]}")   
    

print(f"Final metric: {metric}")

## Plotting the values of the metric_list versus the sample number.

In [ ]:
import matplotlib.pyplot as plt
# Plot the values of the metric_list verus the number of samples
plt.plot(metric_list)
plt.xlabel("Number of samples")
plt.ylabel("MAE")
plt.show()

* Da der `SGD` nicht-deterministisch ist, führt jeder Aufruf zu einem (leicht) modifizierten Ergebnis.

# SVM (ALMA) Classification for Synthetic Data

## Loading the data 

* Synthetic classification data is generated using the `make_classification` function from the `sklearn` package, see [sklearn.datasets.make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html).
  * This function allows the creation of classification problems with $n$ classes.
  * In our example, $n=2$ is chosen (this is the default).  
* The data is then split into the training and test datasets using the `train_test_split` function.

In [ ]:
from sklearn.datasets import make_classification
from river import linear_model

X, y = make_classification(shuffle=True, n_samples=2000)

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


## Model Selection and Model Fit

In [ ]:
model = linear_model.ALMAClassifier()

# fit the model
for x_i,y_i in zip(X_train,y_train):
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    model.learn_one(x_json,y_i)


## Predict on the test set and compute the accuracy

In [ ]:
preds = []
for x_i in X_test:
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    preds.append(model.predict_one(x_json))

# compute accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

## Show the scikit-learn "classification report"

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

# SVM (ALMA) classification for a sklearn classification dataset

In [ ]:
## Generate a classification dataset with sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=10000, n_features=20, n_informative=2, n_redundant=2, random_state=42)

scaler = preprocessing.StandardScaler()


## Convert X to a pandas dataframe
import pandas as pd
X = pd.DataFrame(X)
y = pd.Series(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

* The model is fitted on the training data:

In [ ]:
model = linear_model.ALMAClassifier()
for xi, yi in stream.iter_pandas(X_train,y_train):

    # Before river 0.21.0:
    # Scale the features
    # xi_scaled = scaler.learn_one(xi).transform_one(xi)

    # After 0.21.0:
    scaler.learn_one(xi)
    xi_scaled = scaler.transform_one(xi)

    model.learn_one(xi_scaled, yi) 

*  Prediction on the test data:

In [ ]:
preds = []
for xi, _ in stream.iter_pandas(X_test):    

    # Before river 0.21.0:
    # Scale the features
    # xi_scaled = scaler.learn_one(xi).transform_one(xi)

    # After 0.21.0:
    scaler.learn_one(xi)
    xi_scaled = scaler.transform_one(xi)

    preds.append(model.predict_one(xi_scaled))

* Computation of the accuracy metric:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

# Passiv-Aggressiv Classification for Synthetic Data

In [ ]:
from sklearn.datasets import make_classification
X,y=make_classification(shuffle=True,n_samples=2000)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from river import linear_model

model = linear_model.PAClassifier()

for x_i,y_i in zip(X_train,y_train):
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    model.learn_one(x_json,y_i)
    

preds = []
for x_i in X_test:
    x_json = {'val'+str(i): x for i,x in enumerate(x_i)}
    preds.append(model.predict_one(x_json))

from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

# Classification Tree

In [ ]:
import matplotlib.pyplot as plt
import datetime as dt

from river import datasets
from river import evaluate
from river import metrics
from river import preprocessing
from river.datasets import synth
from river import tree

from river.datasets import synth

dataset = synth.SEA(variant=0, seed=42)
#dataset = datasets.Phishing()
data = dataset.take(10000)

model = tree.HoeffdingTreeClassifier(grace_period=50)

for x, y in data:
    print(x, y)
    x = {f'x_{key}': value for key, value in x.items()}
    print(x,y)
    model.learn_one(x, y)

model

model.summary

model.draw()
